In [1]:
!pip install -U scikit-fuzzy

  Using cached scikit_fuzzy-0.4.2-py3-none-any.whl


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [3]:
temperatura = ctrl.Antecedent(np.arange(-20, 51, 1), 'temperatura')
umidade_ar = ctrl.Antecedent(np.arange(0, 101, 1), 'umidade_ar')
umidade_solo = ctrl.Antecedent(np.arange(0, 101, 1), 'umidade_solo')
nivel_agua = ctrl.Antecedent(np.arange(0, 101, 1), 'nivel_agua')
sistema = ctrl.Consequent(np.arange(12, 36, 1), 'sistema')

sistema.defuzzify_method = 'centroid'

In [4]:
temperatura['muito baixa'] = fuzz.trapmf(temperatura.universe, [-20, -20, -5, 0])
temperatura['baixa'] = fuzz.trimf(temperatura.universe, [-5,10,18])
temperatura['média'] = fuzz.trapmf(temperatura.universe, [15,20,25,28])
temperatura['alta'] = fuzz.trimf(temperatura.universe, [25,28,39])
temperatura['muito alta'] = fuzz.trapmf(temperatura.universe, [39,45,50,50])

umidade_ar['baixa'] = fuzz.trimf(umidade_ar.universe, [0, 0, 30])
umidade_ar['média'] = fuzz.trapmf(umidade_ar.universe, [25, 50, 80, 100])
umidade_ar['alta'] = fuzz.trimf(umidade_ar.universe, [80, 100, 100])

umidade_solo['baixa'] = fuzz.trimf(umidade_solo.universe, [0, 0, 30])
umidade_solo['média'] = fuzz.trapmf(umidade_solo.universe, [25, 50, 80, 100])
umidade_solo['alta'] = fuzz.trimf(umidade_solo.universe, [80, 100, 100])

nivel_agua['baixa'] = fuzz.trapmf(nivel_agua.universe, [0, 0, 30, 40])
nivel_agua['regular'] = fuzz.trimf(nivel_agua.universe, [28, 50, 72])
nivel_agua['alta'] = fuzz.trapmf(nivel_agua.universe, [60, 80, 100, 100])

sistema['resfriar2'] = fuzz.trimf(sistema.universe, [12, 12, 17])
sistema['resfriar1'] = fuzz.trimf(sistema.universe, [15, 18, 20])
sistema['desligar'] = fuzz.trimf(sistema.universe, [19, 22, 25])
sistema['aquecer1'] = fuzz.trimf(sistema.universe, [22, 25, 27])
sistema['aquecer2'] = fuzz.trapmf(sistema.universe, [25, 30, 35, 35])